In [1]:
# Libraries for selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# Waited conditionals for Selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Data Manipulations libraries
import pandas as pd

# Regular expressions library
import re

# Datetime library
import time

In [2]:
# Open the web browser
driver = webdriver.Edge()
# Maximize the window
driver.maximize_window()

In [3]:
# Acceder al SIGMAA
url = 'https://uclb.ucaribe.edu.mx/sigmaav2/'
driver.get(url)

In [4]:
# Biblioteca para leer archivos
import os
# Biblioteca para leer .env
from dotenv import load_dotenv
# Cargamos variables de ambiente
load_dotenv()

# Obtener las credenciales del estudiante
id_user = os.getenv('ID')
password = os.getenv('PASS')
MYSQL_ADDON_URI = os.getenv('MYSQL_ADDON_URI')

In [5]:
# Buscar el campo de usuario y escribir el username
userinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/span[2]/input')
userinput.send_keys(id_user)

In [6]:
# Buscar el campo de contraseña y escribir el password
passinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/input')
passinput.send_keys(password)

In [7]:
# Buscar el botón de submit y dar clic para iniciar sesión
submitinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/button')
submitinput.click()

In [8]:
horarioBoleta = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[4]/a')
horarioBoleta.click()

In [9]:
# Ajustar periodo escolar
periodo = '202001 Primavera'

# Seleccionar una fecha de consulta
periodoEscolar = driver.find_element(By.XPATH, '/html/body/center/form/select')
periodoEscolar.send_keys(periodo)

In [10]:
# Consultar fecha
consultarPeriodo = driver.find_element(By.XPATH, '/html/body/center/form/input[4]')
consultarPeriodo.click()

In [11]:
# Extraer la tabla usando webscrapping
dfs = pd.read_html(driver.page_source)
horario = dfs[1]

In [12]:
# Cambiar nombre de columnas
columnas = ['No.', 'Tipo', 'Seccion', 'Asignatura', 'Primer parcial', 'Segundo parcial', 'Tercer parcial', 'Promedio', 'Calificacion final', 'U1', 'U2']

# Limpieza del dataframe de boleta
# Cambiar nombre de columnas
horario = horario.set_axis(columnas, axis=1)
# Eliminar columnas no deseadas
horario = horario.drop(['U1', 'U2'], axis = 1)
# Eliminar filas con valores nulos
horario.dropna(how = 'all', inplace = True)

In [13]:
horario

,No.,Tipo,Seccion,Asignatura,Primer parcial,Segundo parcial,Tercer parcial,Promedio,Calificacion final
0,1.0,OP1,2.0,DP0498 Problemas de México Alavez San Pedro /...,8.0,7.0,7.0,7.33333349,7.0
1,2.0,OP1,3.0,II0320 Ecuaciones diferenciales Casas De La C...,7.0,6.0,10.0,7.66666651,9.0
2,3.0,OP1,1.0,AD0013 Tenis Morales Oliva / Justino Asignat...,Aprobado,Aprobado,Aprobado,Aprobado,Aprobado
3,4.0,OP1,2.0,ID0205 Geometría computacional Álvarez Estrad...,7.0,8.0,9.0,8.0,8.0
4,5.0,OP1,6.0,IL0204 Estadística analítica Chan Ramayo / Ca...,8.0,10.0,7.0,8.33333302,8.0
5,6.0,OP1,1.0,IT0263 Electrónica analógica René / Mónica Pa...,5.0,5.0,9.0,6.33333349,8.0
6,7.0,OP1,1.0,ID0307 Ingeniería de software Serrano Ruiz / ...,8.0,8.0,4.0,6.66666651,7.0
7,8.0,OP1,1.0,ID3416 Diseño multimedia González Castro / Jo...,6.0,8.0,7.0,7.0,7.0


In [14]:
# Importar biblioteca para conexion a base de datos
from sqlalchemy import create_engine
# Establecer conexion a base de datos
engine = create_engine("mysql+mysqldb://root:@localhost/sigmaa")

In [15]:
# Importar biblioteca para conexion a base de datos
from sqlalchemy import create_engine
# Establecer conexion a base de datos
engine = create_engine(MYSQL_ADDON_URI)

In [16]:
nombre_tabla = 'horario_' + id_user
horario.to_sql(nombre_tabla, con = engine, if_exists="replace")

8

In [17]:
# Cerrar sesión
logout = driver.find_element(By.XPATH, '/html/body/div[2]/div/form/a')
logout.click()

In [ ]:
# Cerrar el navegador
driver.quit()